In [2]:
import pandas as pd
import pymc3 as pm
import theano.tensor as tt

# General Plan

Outcomes of games will be modeled as a stochastic process depending on some latent (or not so latent) characteristics of each team. First, to keep things simple, suppose that each team can be summarized by two numbers: offensive and defensive strengths, OS and DS for short. Then, the number of points scored by any team against any other team is some function of the difference in their respective OS and DS. 

A reasonable function to model this might be something like

$$ score(team~A|team~B) = \theta_0 + \frac{s_{max} - \theta_0 }{1 + e^{-(\alpha + OS(A) - DS(B))/\sigma}} + \hat\epsilon$$

where $\theta_0$ is some intercept term, and $s_{max}$ is an approximate maximum score achieved when one team absolutely dominates the other. Empirically, $s_{max} \sim 70-80$. The parameters $\sigma$ measure the spread over which points accumulate and $\alpha$ is some offset (for example, if teams are exactly matched, I don't expect a team to score zero points...). The machine learning part of this project is to learn OS and DS for each team as well as the parameters $\theta_0, s_{max}, \alpha$, and $\sigma$, based on the outcomes of games. Is this a reasonable model? I don't know, maybe...

Beyond being a fun project, this does have a concrete use case. After inferring team's OS and DS, we can perform simulations of the score differential and total points scored for any particular matchup. This might be useful for people trying to inform their betting strategies for betting on college football games.